In [1]:
import numpy as np
import pandas as pd
from scipy import ndimage
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from tables import *
import os
import operator
import sys
import csv
import h5py
import time
from astropy.time import Time
from astropy import units as u
from numba import jit,njit,int32,float32
from numba.typed import List
from functions_import import lin_fit, intersection, plot_scatter, plot_from_pix, draw_box, read_hess_all_runs, pix_to_xy
from functions_import import get_nn_pix, plot_lin_fit, read_data_sets, read_data_sets_utc,apply_selection_cut, read_az_zen_file
from functions_import import get_velocity,get_mean_brightness,get_cumul_brightness,convert_to_time_in_night,angle_adding_of_tracks
def import_module(filename):
    with open(filename, 'r') as file:
        file_contents = file.read()
    module_object =  exec(file_contents)
    return module_object
import_module("D:\\Masterarbeit ECAP\\Trail finder\\functions_import.py")
typed_nn_pix = List()
for i in range(len(nn_pix)):
    typed_nn_pix_entry = List()
    [typed_nn_pix_entry.append(x) for x in nn_pix[i]]
    typed_nn_pix.append(typed_nn_pix_entry)

Function names are:
flash_geom_x , flash_geom_y
lin_fit(x, m, t)
intersection(lst1, lst2)
plot_scatter(x, y, z)
plot_from_pix(pix, z), uncomment in functions_import.py for title with nrun
draw_box()
plot_lin_fit(track)
read_data_sets(firstrun, lastrun)
apply_selection_cut(dict_high_pixel)
pix_to_xy(trail_pix)
read_az_zen_file(dict_high_pix_keys)
read_hess_all_runs(dict_high_pix_keys)
get_velocity(trail_pix, trail_time)
get_mean_brightness(trail_brightness)
get_cumul_brightness(trail_brightness)
pix_to_xy(trail_pix)
convert_to_time_in_night(azzen_time)
angle_adding_of_tracks(tracklist)
read_data_sets_utc(firstrun, lastrun)
get_nn_pix(size) mit nn_pix, bzw. typed_nn_pix for @jit
Function names are:
flash_geom_x , flash_geom_y
lin_fit(x, m, t)
intersection(lst1, lst2)
plot_scatter(x, y, z)
plot_from_pix(pix, z), uncomment in functions_import.py for title with nrun
draw_box()
plot_lin_fit(track)
read_data_sets(firstrun, lastrun)
apply_selection_cut(dict_high_pixel)
pix_to_xy(trail_pix)
rea

In [2]:
@jit 
def adding_possible_meteorites(pos_met):
    tmp = List(pos_met)
    trash = List()
    if (len(pos_met)==0 or len(tmp)==0):
        trash.append([[-1.],[-1.],[-1.]])
        return tmp, trash
    index_to_del = List()
    for N in range(len(tmp)-1): #Add all trails that are at the same time or just 1s apart from each other
        if len([x for x in tmp[-(N+1)][1] if x in tmp[-(N+2)][1]])>0:
            tmp[-(N+2)][0] = tmp[-(N+2)][0]+ tmp[-(N+1)][0]#pix
            tmp[-(N+2)][1] = tmp[-(N+2)][1]+ tmp[-(N+1)][1]#tmp
            tmp[-(N+2)][2] = tmp[-(N+2)][2]+ tmp[-(N+1)][2]#brightness
            index_to_del.append(-(N+1)+len(tmp))
    for N in range(len(tmp)): #put everything in trash less than 4 unique pixels or longer than 100s(possible tracks?)
        if (len(np.unique(np.array(tmp[N][0])))<4 or len(np.unique(np.array(tmp[N][1])))>100):
            trash.append(tmp[N])
            if N in index_to_del:
                continue
            index_to_del.append(N)
            continue
        for i in np.unique(np.array(tmp[N][0])):#reject
            a = np.array(tmp[N][0])
            count = (a == i).sum()
            if count>8:
                if N in index_to_del:
                    continue
                index_to_del.append(N)
                continue
    index_to_del = sorted(index_to_del)
    for i in range(len(index_to_del)):
        del tmp[index_to_del[-(i+1)]]
    return tmp, trash

In [3]:
@jit
def angle_adding_of_tracks(tracklist):
    track_added = []
    thetas = []
    if tracklist[0][0][0]== -1:
        track_added.append(tracklist[0])
        thetas.append(-1.)
        return track_added
    max_angle_dif = 5.
    for i in range(len(tracklist)):
        x,y = pix_to_xy(tracklist[i][0])
        x_min_time = x[np.where(np.array(tracklist[i][1]) == np.min(np.array(tracklist[i][1])))]
        y_min_time = y[np.where(np.array(tracklist[i][1]) == np.min(np.array(tracklist[i][1])))]
        x_max_time = x[np.where(np.array(tracklist[i][1]) == np.max(np.array(tracklist[i][1])))]
        y_max_time = y[np.where(np.array(tracklist[i][1]) == np.max(np.array(tracklist[i][1])))]
        v = np.array([np.average(x_max_time)-np.average(x_min_time), np.average(y_max_time)-np.average(y_min_time)] )
        if np.sqrt(v.dot(v)) == 0:
            track_added.append(tracklist[0])
            thetas.append(-1.-max_angle_dif)
            continue
        #vector v is calculated between the avg of pixel coordinates of first and latest times respectively
        theta = np.arccos( v[0]/np.sqrt(v.dot(v)) )*180/np.pi
        if i >0:
            if (theta-max_angle_dif<thetas[-1] and theta+max_angle_dif>thetas[-1]):
                max_time_dif = 30.
                if track_added[-1][1][-1]<tracklist[i][1][0]-max_time_dif:
                    #don't add together if max_time_dif is too large 
                    track_added.append(tracklist[i])
                    thetas.append(theta)
                    continue
                track_added[-1][0] = track_added[-1][0]+tracklist[i][0]
                track_added[-1][1] = track_added[-1][1]+tracklist[i][1]
                track_added[-1][2] = track_added[-1][2]+tracklist[i][2]
                thetas.append(theta)
                continue
        track_added.append(tracklist[i])
        thetas.append(theta)
    return track_added

In [4]:
@jit
def angle_adding_of_tracks_List(tracklist):
    track_added = List()
    thetas = []
    if tracklist[0][0][0]== -1:
        track_added.append(tracklist[0])
        thetas.append(-1.)
        return track_added
    max_angle_dif = 5.
    for i in range(len(tracklist)):
        x,y = pix_to_xy(tracklist[i][0])
        x_min_time = x[np.where(np.array(tracklist[i][1]) == np.min(np.array(tracklist[i][1])))]
        y_min_time = y[np.where(np.array(tracklist[i][1]) == np.min(np.array(tracklist[i][1])))]
        x_max_time = x[np.where(np.array(tracklist[i][1]) == np.max(np.array(tracklist[i][1])))]
        y_max_time = y[np.where(np.array(tracklist[i][1]) == np.max(np.array(tracklist[i][1])))]
        v = np.array([np.average(x_max_time)-np.average(x_min_time), np.average(y_max_time)-np.average(y_min_time)] )
        if np.sqrt(v.dot(v)) == 0:
            track_added.append(tracklist[0])
            thetas.append(-1.-max_angle_dif)
            continue
        #vector v is calculated between the avg of pixel coordinates of first and latest times respectively
        theta = np.arccos( v[0]/np.sqrt(v.dot(v)) )*180/np.pi
        if i >0:
            if (theta-max_angle_dif<thetas[-1] and theta+max_angle_dif>thetas[-1]):
                max_time_dif = 30.
                if track_added[-1][1][-1]<tracklist[i][1][0]-max_time_dif:
                    #don't add together if max_time_dif is too large 
                    track_added.append(tracklist[i])
                    thetas.append(theta)
                    continue
                track_added[-1][0] = track_added[-1][0]+tracklist[i][0]
                track_added[-1][1] = track_added[-1][1]+tracklist[i][1]
                track_added[-1][2] = track_added[-1][2]+tracklist[i][2]
                thetas.append(theta)
                continue
        track_added.append(tracklist[i])
        thetas.append(theta)
    return track_added

In [5]:
@jit 
##################Best Sorter yet##################

def track_sorter(dict_pix, dict_tmp, dict_brightness, typed_nn_pixels):#typed_nn_pixels needed to work together with @jit
    if len(dict_pix)==0:
        tracks = List([[[-1.],[-1.],[-1.]]])
        possible_meteorites = List([[[-1.],[-1.],[-1.]]])
        #It is not possible to have neg. values in track, hence if -1, no track was found
        return tracks, possible_meteorites
    pix = list(dict_pix)
    tmp = list(dict_tmp)
    brightness = list(dict_brightness)    
    nn_pixels = list(typed_nn_pixels)
    
####Comments are for jit compatible List(), if this is needed in the future######
#     x = List()
#     y = List()
#     z = List()
#     x.append(float(pix[0]))
#     y.append(float(tmp[0]))
#     z.append(float(brightness[0]))
#     track_zero = List()
#     track_zero.append(x)
#     track_zero.append(y)
#     track_zero.append(z)
#     tracks = List()
#     tracks.append(track_zero)
#     x = List(float(pix[0]))
#     y = List(float(tmp[0]))
#     z = List(float(brightness[0]))
#     print(x,y,z)
#     track_zero = List([x,y,z])
#     x_met = List()
#     y_met = List()
#     z_met = List()
#     x_met.append(float(pix[0]))
#     y_met.append(float(tmp[0]))
#     z_met.append(float(brightness[0]))
#     track_zero_met = List()
#     track_zero_met.append(x)
#     track_zero_met.append(y)
#     track_zero_met.append(z)
#     possible_meteorites = List()
#     possible_meteorites.append(track_zero_met)
####Comments are for jit compatible List(), if this is needed in the future######
    tracks = List([ [[float(pix[0])],[float(tmp[0])], [float(brightness[0])]]])
    
#     tracks = List([tracks_zero])
    for i in range(1, len(pix)):
        appended_to_track = False
        for N in range(len(tracks)):
            
            if tmp[i]-5 < tracks[N][1][-1]:
                if len([x for x in nn_pixels[pix[i]] if x in tracks[N][0][-20:]])>0:
                    tracks[N][0].append(pix[i])
                    tracks[N][1].append(tmp[i])
                    tracks[N][2].append(brightness[i])
                    appended_to_track  = True 
                    break 
        if appended_to_track == False:
            tracks.append([[float(pix[i])],[float(tmp[i])], [float(brightness[i])]])

    possible_meteorites = List([[[0.],[0.],[0.]]])
    to_remove = np.full((1,len(tracks)), -1)[0]   
    for N in range(len(tracks)):        
        if len(np.unique(np.array(tracks[N][0])))<4:
            possible_meteorites.append(tracks[N])
            to_remove[N] = N
        elif len(np.unique(np.array(tracks[N][1])))<3:
            possible_meteorites.append(tracks[N])
            to_remove[N] = N
    del possible_meteorites[0]
    if len(possible_meteorites)==0:
        possible_meteorites = List([[[-1.],[-1.],[-1.]]])
    
    to_remove = to_remove[to_remove != -1]
    for i in range(len(to_remove)):
        del tracks[to_remove[-(i+1)]]
    if len(tracks)==0:
        tracks = List([[[-1.],[-1.],[-1.]]])
    return tracks, possible_meteorites

In [6]:
def convert_to_time_in_night(azzen_time):
    t = Time(azzen_time, format = 'unix', scale='utc')
    t.format = "iso"
    a = Time(t, format = "iso",out_subfmt = "date_hms")
    b = t.to_value("iso",subfmt = "date")
    b = Time(b, format = "iso", scale = "utc")
    azzen_time_in_day = np.array((a-b).value*3600*24).astype(int)
    return azzen_time_in_day

In [7]:
def increase_threshold(dict_high_pix):
    high_pix_cut = {}
    for nrun in dict_high_pix:
        high_pix_cut[nrun] = {}
        values, counts = np.unique(dict_high_pix[nrun]["pix"], return_counts=True)
        i = 0
        mask = dict_high_pix[nrun]["pix"]>-1
        while len(values)>200:
            mask = dict_high_pix[nrun]["brightness"]>900+i*50
            values, counts = np.unique(dict_high_pix[nrun]["pix"][mask], return_counts=True)
            print(len(values), 900+i*50)
            i+=1
        high_pix_cut[nrun]["pix"] = dict_high_pix[nrun]["pix"][mask]
        high_pix_cut[nrun]["time"] = dict_high_pix[nrun]["time"][mask]
        high_pix_cut[nrun]["brightness"] = dict_high_pix[nrun]["brightness"][mask]
    return high_pix_cut

In [8]:
#remove tracks that are too short in unique pixel length AFTER ADDING TOGETHER
@jit 
def cut_on_pix(tracklist):
    track_cut = List()
    if tracklist[0][0][0]== -1.:
        track_cut.append(tracklist[0])
        return track_cut
    min_pix_len = 4.
    for i in range(len(tracklist)):
        if np.unique(tracklist[i][0])>= min_pix_len:
            track_cut.append(tracklist[i])
    if len(track_cut)==0:
        track_cut.append([[-1.0], [-1.0], [-1.0]])
    return track_cut

In [9]:
#remove tracks that are too short in time difference between first and last entry AFTER PIXEL CUT
@jit
def cut_on_time(tracklist):
    track_cut = List()
    if tracklist[0][0][0]== -1.:
        track_cut.append(tracklist[0])
        return track_cut
    min_time_dif = 4.
    for i in range(len(tracklist)):
        if tracklist[i][1][-1]-tracklist[i][1][0] >= min_time_dif:
            track_cut.append(tracklist[i])
    if len(track_cut)==0:
        track_cut.append([[-1.0], [-1.0], [-1.0]])
    return track_cut

In [ ]:
%%time
#######################################################
########example evaluation of data:####################
#######################################################
# print(os.getcwd())
high_pix = read_data_sets(155200, 175000)#Numbers divisible by 200 are needed
hessall_nruns, hessall_i = read_hess_all_runs(high_pix.keys()) #gives all runs in selected range that are also in hessall.txt
keylist = list(high_pix.keys())
for nrun in keylist:
    if nrun not in hessall_nruns:
        del high_pix[nrun]
az_zen_nruns, az_zen_az, az_zen_zen, az_zen_utc, az_zen_time = read_az_zen_file(high_pix.keys())#gives az, zen, etc. for each run 
                                                                                                #h might need to change 
                                                                                                #to in hessall_nruns
high_pix = apply_selection_cut(high_pix, az_zen_time) #cut away each pixel with more than 70 entries
                                         #az_zen_time needed to properly adjust the starting time in a run 
high_pix = increase_threshold(high_pix)
az_zen_time_in_day = convert_to_time_in_night(az_zen_time) #gives starting time in the night in seconds after midnight

#Declaration of lists that will be evaluated by histograms later on
track_nrun = []
track_N_nrun = []
track_time_absolute = []
track_time_in_night = []
track_az = []     #these 3 will be used for 2d histogram
track_zen = []

velocities = []
mean_brightnesses = []
cumulative_brightnesses = []
duration_on_cam = []
unique_pix_in_track = []
nn_pix = get_nn_pix(7)
#Loop over all nruns in dictionary high_pix:
az_nrun = 0 #extra declaration needed to access az_zen_... arrays
print(len(high_pix.keys()))
print(len(az_zen_time_in_day))
print(len(az_zen_time))
print(len(az_zen_az))
print(len(az_zen_zen))
for nrun in high_pix:
    if len(high_pix[nrun]["pix"]) == 0:
        az_nrun +=1 
        continue
    if np.max(high_pix[nrun]["time"])<10:
        az_nrun +=1 
        continue
    
    track, possible_meteorite = track_sorter(high_pix[nrun]["pix"],
                                         high_pix[nrun]["time"],
                                         high_pix[nrun]["brightness"],
                                        typed_nn_pix)
    #Sort into possible tracks and the rest, further cleaning/removal is required
    possible_meteorite, trash = adding_possible_meteorites(possible_meteorite)  #adds close times toghether, but still has 
                                                                                    #unwanted things in pos_met
    track = angle_adding_of_tracks(track)
    
    print(nrun, len(track), len(possible_meteorite))
    for i in range(len(track)):
        if track[i][0][0] == -1:
            continue
        v = get_velocity(np.array(track[i][0]), np.array(track[i][1]))
        b = get_mean_brightness(np.array(track[i][2]))
        cb = get_cumul_brightness(np.array(track[i][2]))
        velocities.append(v)
        mean_brightnesses.append(b)
        cumulative_brightnesses.append(cb)
        duration_on_cam.append(track[i][1][-1]-track[i][1][0])
        unique_pix_in_track.append(len(np.unique(track[i][0])))
        
        x, y = pix_to_xy(track[i][0]) #Coordinates from pixelids
        popt, pcov = curve_fit(lin_fit, x,y) #get linear fit parameters from each run. Strange results for vertical trails
        first_time = track[i][1][0]
        if int(first_time)>100000:
            first_time = 0
        fig = plt.figure()
        track_time_in_night.append(az_zen_time_in_day[az_nrun]+int(first_time))
        track_time_absolute.append(az_zen_time[az_nrun]+int(first_time))
        track_az.append(az_zen_az[az_nrun])
        track_zen.append(az_zen_zen[az_nrun])
        track_nrun.append(nrun)
        track_N_nrun.append(len(track))
        plt.scatter(x,y, c = track[i][1], cmap = "jet")
        plt.plot(x, lin_fit(x, popt[0], popt[1]))
        draw_box()
        plt.colorbar()
        plt.show()
        plt.close(fig)
    az_nrun +=1 
    
#plot Histograms:
plt.hist(velocities, 40)
plt.title("Velocities")
plt.show()
plt.hist(mean_brightnesses, 40)
plt.title("mean_brightnesses")
plt.show()
plt.hist(cumulative_brightnesses, 40)
plt.title("cumulative_brightnesses")
plt.show()
plt.hist2d(track_zen,track_time_in_night)
plt.title("zenith and time in night")
plt.show()

D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155200-155399
Reading high_pix_155200_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155200-155399
No file named high_pix_155201_CT_5.h5 found in folder 155201
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155200-155399
No file named high_pix_155202_CT_5.h5 found in folder 155202
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155200-155399
Reading high_pix_155203_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155200-155399
No file named high_pix_155204_CT_5.h5 found in folder 155204
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155200-155399
Reading high_pix_155205_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155200-155399
No file named high_pix_155206_CT_5.h5 found in folder 155206
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run1

D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155200-155399
Reading high_pix_155396_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155200-155399
Reading high_pix_155397_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155200-155399
Reading high_pix_155398_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155200-155399
Reading high_pix_155399_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155400-155599
Reading high_pix_155411_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155400-155599
No file named high_pix_155412_CT_5.h5 found in folder 155412
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155400-155599
Reading high_pix_155413_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155400-155599
No file named hig

Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155800-155999
Reading high_pix_155982_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run155800-155999
Reading high_pix_155983_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156000-156199
Reading high_pix_156026_CT_5.h5
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156000-156199
Reading high_pix_156027_CT_5.h5
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156000-156199
Reading high_pix_156028_CT_5.h5
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156000-156199
Reading high_pix_156029_CT_5.h5
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156000-156199
Reading high_pix_156030_CT_5.h5
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156000-156199
No file named high_pix_156060_CT_5.h5 found in folder 156060
D:\Masterarbeit ECAP\Trail finder\eval_hig

D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156400-156599
Reading high_pix_156403_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156400-156599
Reading high_pix_156404_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156400-156599
Reading high_pix_156405_CT_5.h5
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156400-156599
Reading high_pix_156406_CT_5.h5
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156400-156599
Reading high_pix_156407_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156400-156599
Reading high_pix_156422_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156400-156599
Reading high_pix_156423_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156400-156599
Reading high_pix_156424_CT_5.h5
Reading successful
D:\Masterarbei

Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156800-156999
Reading high_pix_156928_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156800-156999
Reading high_pix_156929_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156800-156999
Reading high_pix_156930_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156800-156999
Reading high_pix_156931_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156800-156999
Reading high_pix_156959_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156800-156999
Reading high_pix_156960_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156800-156999
Reading high_pix_156961_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run156800-156999
No file 

Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157200-157399
Reading high_pix_157341_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157200-157399
Reading high_pix_157342_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157200-157399
Reading high_pix_157343_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157200-157399
Reading high_pix_157344_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157200-157399
Reading high_pix_157345_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157200-157399
Reading high_pix_157346_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157200-157399
Reading high_pix_157347_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157200-157399
No file 

Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading high_pix_157428_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading high_pix_157429_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading high_pix_157430_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading high_pix_157431_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading high_pix_157432_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading high_pix_157433_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading high_pix_157434_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading 

Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading high_pix_157529_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading high_pix_157530_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading high_pix_157531_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading high_pix_157532_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading high_pix_157533_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading high_pix_157568_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading high_pix_157569_CT_5.h5
Reading successful
D:\Masterarbeit ECAP\Trail finder\eval_high_data\high_pixel\run157400-157599
Reading 